# Interactive VAPD exploration

This notebook is meant to help exploring the properties of the VAPD.

Scroll down, past the initial code, to Example 1

In [1]:
import numpy as np
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression

In [2]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.text_cell_render { font-size: 18pt; }</style>'))

In [4]:
import matplotlib as mpl
mpl.rcParams['figure.figsize']=(10,8)

In [5]:
from ipywidgets import interact,FloatSlider,Layout

In [6]:
from statsmodels.distributions.empirical_distribution import ECDF
def showPredDistrib(x_test):
    f,ax = plt.subplots(1,3,figsize=(10,6))
    ax[0].plot(X,y,".")
    ax[0].plot(X,s)
    ax[0].vlines(x_test,ax[0].get_ylim()[0],ax[0].get_ylim()[1],'g')
    ax[0].set_xlabel("$x$")
    ax[0].set_ylabel("$y,s(x)$")

    s_test = clf.predict(np.atleast_2d(x_test)).ravel()

    ax[1].plot(s,g,",")
    ax[1].set_xlabel("$s$")
    ax[1].set_ylabel("$g(s)$")    
    ax[1].vlines(s_test,ax[1].get_ylim()[0],ax[1].get_ylim()[1],'g')

    g_test = ir.predict(s_test)
    ys = y[g== g_test].ravel()
    if len(ys)!=1:
        ecdf = ECDF(ys)
        ax[2].plot(ecdf.x,ecdf.y)
    ax[2].set_xlim([-5,4])
    ax[2].set_xlabel("$t$")
    ax[2].set_ylabel("$P[y \leq t]$")
    
    f.tight_layout()
    
    # f.setsize_inches(8,4)

# Example 1 - Homoscedastic data

Data $(x_i,y_i)$ corresponding to a straight line with slope -2 with additive Gaussian noise is generated.

A linear regression model is fitted obtaining predictions $s_i$

An Isotonic Regression between $s$ and $y$ is fitted obtaining predictions $g_i$.

In [7]:
np.random.seed(101)
X = np.linspace(0,1,1001).reshape(-1,1)
y = -2*X+np.random.normal(size=X.shape)

In [8]:
clf = LinearRegression()
clf.fit(X,y)
s = clf.predict(X)
# s = y

ir = IsotonicRegression(increasing=True)
ir.fit(s.ravel(),y.ravel())

g = ir.predict(s.ravel())

Below there is an interactive plot, with a slider where one can set the value of the $x_{test}$.

The left plot show the data (blue) and the regression (red); the vertical green line correspond to the test point selected by the slider.

The middle plot is the Isotonic Regression $g(s)$; the vertical line is the $s(x)$ corresponding to the test point.

The right plot is the predictive distribution of $y$, calculated as ECDF $F_Y(t) = \mathbb P[Y \leq t]$ on the element of the taxonomy induced by the Isotonic Regression.
(The taxonomy consists of the sets of $x$ with the same value of $g(s(x))$)

In [9]:
interact(showPredDistrib,x_test = FloatSlider(value=0.5,min=0,max=1,step=0.005,layout=Layout(width='50%')));

interactive(children=(FloatSlider(value=0.5, description='x_test', layout=Layout(width='50%'), max=1.0, step=0.005), Output()), _dom_classes=('widget-interact',))

One obvious downside of this taxonomy is that there are small categories. In some cases, the categories contain one element only.

# Heteroscedastic data

The following shows that the predictive distribution is specific and captures a variation of variance in the data.

In [10]:
X = np.linspace(0,1,1001).reshape(-1,1)
y = -2*X+np.random.normal(size=X.shape)*(X*1+0.5)

clf = LinearRegression()
clf.fit(X,y)
s = clf.predict(X)
# s = y

ir = IsotonicRegression(increasing=True)
ir.fit(s.ravel(),y.ravel())

g = ir.predict(s.ravel())

In [11]:
interact(showPredDistrib,x_test = FloatSlider(value=0.5,min=0,max=1,step=0.005,layout=Layout(width='50%')));


interactive(children=(FloatSlider(value=0.5, description='x_test', layout=Layout(width='50%'), max=1.0, step=0.005), Output()), _dom_classes=('widget-interact',))

# Non linear data


In the cases above, the dependent variable had a linear relation. That made $g()$ be ideally $g(x)=x$. This seemed to justify the choice of a monotonic $g()$.

What if the error between $s(x)$ and $y(x)$ changed "direction" for different values of $s$?

The mapping g() that reduces the error may no longer be monotonic.

I try to create such a situation below.

Observe the shape of $g()$

Is this desirable behaviour?

In [12]:
X = np.linspace(0,1,1001).reshape(-1,1)
y = np.sin(np.pi*X)+0.2*np.random.normal(size=X.shape)

clf = LinearRegression()
clf.fit(X,y)
s = clf.predict(X)
# s = y

ir = IsotonicRegression(increasing=True)
ir.fit(s.ravel(),y.ravel())

g = ir.predict(s.ravel())

In [13]:
interact(showPredDistrib,x_test = FloatSlider(value=0.5,min=0,max=1,step=0.005,layout=Layout(width='50%')));

interactive(children=(FloatSlider(value=0.5, description='x_test', layout=Layout(width='50%'), max=1.0, step=0.005), Output()), _dom_classes=('widget-interact',))